In [300]:
import sys
!{sys.executable} -m pip install bs4
import requests
from pandas.io.json import json_normalize
import json
from datetime import datetime
from datetime import timedelta
import feedparser
import pandas as pd

In [372]:
def scrapping(soup):
    #Obtengo los principales índices para el cierre anterior (es una tabla)
    tabla_indices = soup.select("table table table table table th")
    # tabla_indices
    headers = [element.text for element in tabla_indices[0:10]]

    # Guardo los nombres de la columna para la tabla
    headers

    #Obtengo todos los datos que se encuentran en la tabla, en los tabledata
    datos  = soup.select("table table table table table td")

    #Tomo el texto de cada elemento y los guardo, sin sus etiquetas
    datos = [element.text for element in datos]

    #Creo una lista de listas donde guardaré los valores para cada fila que luego pasaré al Data Frame
    filas = [[]]

    #Hago un slicing por cada 10 elementos que son los valores para cada columna por fila
    for num in range(0,71, 10):
        filas.append(datos[num:num+10])

    df = pd.DataFrame(filas, columns = headers)
    return df
#     print(df)

#Obtengo el RSS de los cierres diarios de la BMV
def scrapping_RSS():
    url = "http://feeds.feedburner.com/BmvBoletin"

    rss = feedparser.parse(url)

    #Guardo cada una de las entradas existentes en el RSS
    rss_df = pd.DataFrame(rss.entries)
#     print(rss_df)

    #Ordeno las entradas para tener en el primer elemento la más reciente (el día anterior)
    rss_df_sorted = rss_df
    rss_df_sorted["published"] = rss_df_sorted["published"].astype("datetime64[ns]")
    rss_df_sorted = rss_df.sort_values("published", ascending = False)

    #Obtengo el cierre anterior que es un archivo HTML
    last_closed_link = rss_df_sorted["feedburner_origlink"][len(rss_df_sorted) - 1]

    #Leo el archivo HTML del cierre del día anterior de la BMV
    html_file = requests.get(last_closed_link).content

    soup = BeautifulSoup(html_file)

    return scrapping(soup)


#######Funciones sobre el DataFrame######

def drop_duplicates(df):
    df = df.drop_duplicates()
    return df

def drop_na_rows(df):
    df = df.dropna()
    return df

def drop_na_columns(df):
    null_cols = df.isnull().sum()
    drop_cols = list(null_cols[null_cols > 10000].index)
    df = df.drop(drop_cols, axis=1)
    return df 

def data_cleaning(df):
    df = drop_duplicates(df)
    df = drop_na_rows(df)
    df = drop_na_columns(df)
    return df
    

#####Cálculo de información importante
def variacion_negativa(df):
    df["Variación"] = df["Variación"].astype(float)
    negativos = df[df["Variación"] < 0]
    return negativos["Índice", "Último", "Hora", "Anterior", "Variación", "Variación Ptos.", "Máximo", "Hora", "Mínimo", "Hora"]
    
#####Cálculo de variación negativa en puntos
def variacion_negativa_puntos(df):
    df["Variación Ptos."] = df["Variación Ptos."].astype(float)
    negativos = df[df["Variación Ptos."] < 0]
    return negativos["Índice", "Último", "Hora", "Anterior", "Variación", "Variación Ptos.", "Máximo", "Hora", "Mínimo", "Hora"]
    
def export_to_csv(df):
    df.to_csv("bmv.csv")    

df = scrapping_RSS()
df = data_cleaning(df)
export_to_csv(df)

#Obtener los índices con variación negativa
variacion_negativa(df)
variacion_negativa_puntos(df)






Índice     Último   Hora   Anterior  Variación  \
1              S&P/BMV IPC  41,618.11  15:16  42,551.54      -2.19   
2         S&P/BMV INMEX RT   3,068.16  15:16   3,143.06      -2.38   
3  S&P/BMV IPC SUSTENTABLE     273.36  15:16     279.79      -2.30   
4           S&P/BMV FIBRAS     218.96  15:16     219.53      -0.26   
5       S&P/BMV IRT CompMx     449.31  15:16     459.22      -2.16   
6     S&P/BMV IRT LargeCap     472.43  15:16     482.92      -2.17   
7       S&P/BMV IRT MidCap     336.65  15:16     345.01      -2.42   
8     S&P/BMV BURSA ÓPTIMO     172.88  15:16     177.00      -2.33   

   Variación Ptos.     Máximo  Hora     Mínimo  Hora  
1          -933.43  42,541.54  8:30  41,429.12  2:38  
2           -74.90   3,141.99  8:30   3,056.01  2:39  
3            -6.43     279.69  8:30     272.44  2:39  
4            -0.57     220.35  8:43     217.28  1:08  
5            -9.91     459.04  8:30     447.62  2:39  
6           -10.49     482.80  8:30     470.91  2:39  
7            -8.36     345.21  8:56     334.71  2:39  
8            -4.12     176.96  9:22     172.10  2:39

API CHALLENGE

In [370]:
#Función que utiliza la Api de la Nasa para obtener los asteroides que se encuentran cerca de la tierra
#En una fecha específica mostrando algunos datos importantes de cada uno de ellos
def get_response(start_date):
    start_date_api = datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = start_date_api + timedelta(days = 7)
    
    #En los parámetros de la API se encuentra la fecha de inicio y fecha de fin, para obtener los datos de los asteroides
    #Por día entonces obtengo el día final para enviarlo como parámetro por ser requerido pero al final no lo uso más que 
    #para el cálculo
    
    url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date_api}&end_date={end_date}&api_key=T0tZNACEDXLIyexmdmTCSCrMemQ0u39v1syrwXzJ"
    response = requests.get(url)
    asteroids_json = response.json()
    asteroids_normalized = json_normalize(asteroids_json)
    asteroids_df = pd.DataFrame(asteroids_normalized[f"near_earth_objects.{start_date}"][0])
    
    #Una vez obtenida la tabla, elijo solo las columnas que considero que contienen la información importante
    asteroids_return_df = asteroids_df[["name", "neo_reference_id", "is_potentially_hazardous_asteroid", "absolute_magnitude_h"]]
    return asteroids_return_df

#Esta función lo que hace es contar cuántos de todos los asteroides presentan un peligro real para la tierra
def count_hazardous(df):
    contador = 0
    for element in df["is_potentially_hazardous_asteroid"]:
        if element == False:
            contador += 1
    print(contador)
    
def export_to_csv(df):
    df.to_csv("asteroids.csv")

df = get_response("2019-07-12")  
count_hazardous(df)
export_to_csv(df)

11
